In [ ]:
%pylab notebook

# Fetch the evaluation / holdout data

In [ ]:
eval_txt = 'data/training/independant_12_layers/fold_01/eval.txt'
eval_files = [fn.strip() for fn in open(eval_txt).readlines()]
print "Found {} eval files".format(len(eval_files))

In [ ]:
cmp_eval_files = [f for f in eval_files if '/cmp/' in f]
print "Found", len(cmp_eval_files), "from the CMP dataset", len(eval_files)-len(cmp_eval_files), "are our own"

In [ ]:
def show_images(images, nrows=4, ncols=4):
    for i in range(nrows*ncols):
        subplot(nrows, ncols, i+1)
        labeled_data = np.load(images[i])
        rgb = labeled_data[:3]
        features = labeled_data[3:]
        window  = features[1]
        imshow(rgb.transpose(1,2,0)/255.)
        xticks([]); yticks([])

In [ ]:
from pyfacades.util.metrics import Metrics

In [ ]:
LABELS= ['background', 'facade', 'window', 'door', 'cornice', 'sill',
         'balcony', 'blind', 'deco', 'molding', 'pillar', 'shop']

In [ ]:
m = Metrics()

In [ ]:
!nvidia-smi

In [ ]:
import caffe
caffe.set_mode_cpu()
# caffe.set_mode_gpu()
# caffe.set_device(0)

In [ ]:
PROTO = 'non-bayesian-inference-net.prototxt'
#WEIGHTS = 'deploy/test_weights.caffemodel'
WEIGHTS = 'test_weights_from_peihao.caffemodel'
net = caffe.Net(PROTO, WEIGHTS, caffe.TEST)

# Set the batch size to one -- we are using the CPU
net.blobs['data'].reshape(1, 3, 512, 512)
net.reshape()

The F1 score will depend on the amount of dilatin. Right now it is hardcoded as 3. Should search for best value...

In [ ]:
NEG = 0
UNK = 1
POS = 2
EDG = 3

In [ ]:
LABELS

In [ ]:
WINDOW = LABELS.index('window')
BACKGROUND=LABELS.index('background')

f = cmp_eval_files[9]

def get_metrics(f):
    blob = np.load(f)
    rgb = blob[:3]
    expected = blob[3:]  
    expected_windows = expected[WINDOW]
    expected_windows[expected[BACKGROUND]==2] =1 # ignore
    
    # Remove padding from considereation
    padding = rgb.max(axis=0).max(axis=0) == 0
    padding = np.outer(np.ones(rgb.shape[1]), padding) > 0
    expected_windows[padding] = 1
  
    net.forward(data=np.array([rgb]))
    predicted_windows = net.blobs['label-window'].data[0].squeeze()

    mf = Metrics(expected=expected_windows, 
                 predicted=predicted_windows, 
                 label_positive=POS,
                 label_negative=NEG,
                 source=f,
                 feature='window'
                )
    return mf, rgb, expected_windows, predicted_windows

figure()
mf, rgb, expected, predicted = get_metrics(f)
imshow(net.blobs['prob-window'].data[0, 2])

In [ ]:
import anydbm
import json
import hashlib
import munch

In [ ]:
eval_results = anydbm.open('eval_low_rank_peihao', 'c')
checksum = hashlib.md5(open(WEIGHTS).read()).hexdigest()

In [ ]:
if 'md5' in eval_results and eval_results['md5'] == checksum:
    print "We already seem to have run evaluation..."

In [ ]:
fig = figure(figsize=(12,4))

recompute = False
visualize = True

total = Metrics(feature='windows')
for i, f in enumerate(eval_files):   
    if recompute or f not in eval_results:
        mf, rgb, expected, predicted = get_metrics(f)
        eval_results[f] = json.dumps(mf.as_dict())
        if visualize:
            visualize_eval(rgb, expected, predicted)
            try:
                suptitle('{} of {}, $P$:{:.2%}, $R$:{:.2%}, $F_1$:{:.2%}, $A$:{:.2%}'.format(i, len(eval_files), total.pixel_precision, total.pixel_recall, total.pixel_f1, total.pixel_accuracy))
            except ZeroDivisionError:
                suptitle("Not enough samples yet....")
                
            fig.canvas.draw()
    else:
        mf = Metrics(**json.loads(eval_results[f]))
    #print mf, 
    total += mf
    #print total
    print '\r{:3} of {}'.format(i, len(eval_files)),


In [ ]:
def reverse_engineer_acc(P, R):
    pos = 5988721
    neg = 57149895
    TP = R*pos
    FN = pos - TP
    FP = TP*(1.-P)/P
    TN = neg-FP
    acc = (TP+TN)/(TP+FP+TN+FN)
    return acc

In [ ]:
print '{:.1%}'.format(reverse_engineer_acc(0.73, 0.62))
print '{:.1%}'.format(reverse_engineer_acc(0.95, 0.69))
print '{:.1%}'.format(reverse_engineer_acc(0.91, 0.71))


In [ ]:
cmp_total = Metrics()
for f in eval_results:
    if '/cmp/' in f:
        #print  f
        cmp_total += Metrics(**json.loads(eval_results[f]))
print cmp_total, 'Acc:', cmp_total.pixel_accuracy

# Find the top scores on CMP and GSV

In [ ]:
#%%file color_coded_errors.py

ERROR_COLORS = [[128, 128,  128], #TN
                [255, 0,    0],   #FP
                [0,   255,  0],   #TP
                [0,   0,    255], #FN
                [0,   0,    0], #ignored
               ]  
ERROR_COLORS = array(ERROR_COLORS, dtype=np.uint8)

def color_coded_errors(expected, predicted, ignored=None, colors=ERROR_COLORS):
    TP = ~ignored & (expected & predicted)
    FN = ~ignored & (expected & ~predicted)
    FP = ~ignored & (~expected & predicted)
    TN = ~ignored & (~expected & ~predicted)
    errors = np.argmax(array([TN, FP, TP, FN, ignored], dtype=np.uint8), axis=0)
    if colors is not None:
        return np.ma.masked_array(colors[errors], np.dstack([TN]*3))
    else:
        return errors  
      
def render_errors(path, alpha=0.5, noFN=True):
    mf, rgb, expected, predicted = get_metrics(path)
    ignored = (expected != mf.label_positive) & (expected != mf.label_negative)
    expected = expected==mf.label_positive
    predicted = predicted==mf.label_positive
    print mf.label_positive
    cc = color_coded_errors(expected, predicted, ignored)
    rgb = rgb.transpose(1,2,0)/255.
    if noFN:
        rgb[~cc.mask] = (1-alpha)*rgb[~cc.mask] + alpha*cc[~cc.mask]
        return rgb.clip(0,1)
        #imshow(predicted)
    else:
        rgb = (1-alpha)*rgb + alpha*cc
    return rgb

In [ ]:
def score(f):
    acc = Metrics(**json.loads(eval_results[f])).pixel_f1
    if isnan(acc):
        acc = 0
    return acc

In [ ]:
accs = array([score(f) for f in eval_files])

cmp_accs = array([accs[i] for i in range(len(eval_files))  if '/cmp/' in eval_files[i]])
cmp_files = array([eval_files[i] for i in range(len(eval_files))  if '/cmp/' in eval_files[i] ])

gsv_accs = array([accs[i] for i in range(len(eval_files)) if '/cmp/' not in eval_files[i] ] )
gsv_files = array([eval_files[i] for i in range(len(eval_files))  if '/cmp/' not in eval_files[i] ])

gsv_ranking = argsort(gsv_accs)
cmp_ranking = argsort(cmp_accs)

In [ ]:
figure()
plot(gsv_accs[gsv_ranking])

In [ ]:
%pdb off

In [ ]:
!nvidia-smi

In [ ]:
cmp_comparison_files = [cmp_files[cmp_ranking[-i-1]] for i in range(9)]

In [ ]:
f = cmp_comparison_files[0]

In [ ]:
f

In [ ]:
gsv_comparison_files = [gsv_files[gsv_ranking[-i-1]] for i in range(9)]
gsv_comparison_files[8]

In [ ]:
data = np.load(gsv_comparison_files[2])
rgb = data[:3]

figure()
imshow(rgb.transpose(1,2,0)/255)
imsave('gsv-example.png', rgb.transpose(1,2,0)/255)

In [ ]:
results = net.forward(data=array([rgb]))

In [ ]:
figure()
imshow(rgb.transpose(1,2,0)/255)

In [ ]:
colors = cm.nipy_spectral(linspace(0, 1, 13))[1:]

In [ ]:
from skimage.morphology import binary_closing, binary_opening

In [ ]:
def color_all(expected=None):
    cc = net.blobs['data'].data[0].transpose(1,2,0)/255
    labels = np.zeros(cc.shape[:2], dtype=int)
    for i, label in enumerate(LABELS):
        if i == 0:
            continue
        if expected is not None:
            mask = expected[i] == POS
        else:
            mask = net.blobs['label-'+label].data[0,0] == POS
        #mask = binary_closing(mask, selem=ones((5, 5)))
        #mask = binary_opening(mask, selem=ones((5, 5)))
        cc[mask] = colors[i][:3]
        labels[mask] = i
    return cc

In [ ]:
from skimage.measure import regionprops

In [ ]:
for i in range(9):
    data = np.load(gsv_comparison_files[i])
    rgb = data[:3]
    results = net.forward(data=array([rgb]))

    figure(figsize=(8, 3))
    subplot(131)
    imshow(rgb.transpose(1,2,0)/255)
    axis('off')
    subplot(132)
    imshow(rgb.transpose(1,2,0)/255)
    imshow(color_all(), alpha=0.75)
    axis('off')
    subplot(133)
    imshow(rgb.transpose(1,2,0)/255)
    imshow(color_all(data[3:]), alpha=0.75)
    axis('off')
    tight_layout()
    subplots_adjust(wspace=0.,hspace=0.)


In [ ]:
figure(figsize=(2, 6))
ax = gca()
w = 0.2
h = 0.05
for i, label in enumerate(LABELS):
    x = 0
    y = i*h
    R = Rectangle((x,y), w, h, fill=True, color=colors[i], alpha=1)
    ax.add_patch(R)
    text(x+w/2, y+h/2, label, horizontalalignment="center", verticalalignment="center")
ylim(0, len(LABELS)*h)
xlim(0,w)
axis('off')

In [ ]:
B = R.get_bbox()

In [ ]:
B.